In [1]:
from bs4 import BeautifulSoup
import re
from bs4 import Tag, NavigableString
from bs4.formatter import HTMLFormatter

In [2]:
with open(r"D:\Users\Dell\Documents\Web Projects\react\Plant Diseases\web-scraping\initial-extract\vegetable crops\mint.html",encoding="UTF-8") as fh:
    soup=BeautifulSoup(fh.read(),"html.parser")

In [3]:
d=soup.find("div",{"class":"col text-right"})

In [4]:
d.decompose()

In [5]:
soup.find("div",{"class":"col text-right"})

In [6]:
removable=soup.find_all("p")[-3:]
removable.append(soup.find_all("h3")[-1])

In [7]:
for tag in removable:
    tag.decompose()

In [8]:
def update_html(soup):
    with open("test.html","w",encoding="UTF-8") as fh:
        fh.write(str(soup.prettify(formatter=HTMLFormatter(indent=4))))

In [9]:
update_html(soup)

In [10]:
def CleanSoup(soup):
    content=clone_soup(soup)
    for tag in content.findAll(True): 
        if tag.name=='a':
            if tag.attrs.get("name",None):
                tag.attrs = {"name":tag.attrs["name"]}
            elif tag.attrs.get("href",None):
                tag.attrs = {"href":tag.attrs["href"]}
            else:
                tag.decompose()
        elif tag.name=='img':
            attrs=dict()
            for prop in ["width","height","alt","src"]:
                value=tag.attrs.get(prop,None)
                if value:
                    attrs[prop]=value
            tag.attrs=attrs
        else:
            tag.attrs = {}
    return content

In [11]:
def clone_soup(el):
    if isinstance(el, NavigableString):
        return type(el)(el)

    copy = Tag(None, el.builder, el.name, el.namespace, el.nsprefix)
    # work around bug where there is no builder set
    # https://bugs.launchpad.net/beautifulsoup/+bug/1307471
    copy.attrs = dict(el.attrs)
    for attr in ('can_be_empty_element', 'hidden'):
        setattr(copy, attr, getattr(el, attr))
    for child in el.contents:
        copy.append(clone_soup(child))
    return copy

In [12]:
update_html(CleanSoup(soup))
soup1=CleanSoup(soup)

In [13]:
for tag in soup1.findAll(True): 
    if tag.name=='a':
        if tag.attrs.get("name",None):
            tag.attrs = {"name":tag.attrs["name"]}
            tag.insert_before(soup.new_string("REPLACE"))

In [14]:
update_html(soup1)

In [15]:
pattern=r'REPLACE(?=<a.*?name="section.*?(?=a>))'

In [16]:
new_text=re.sub(pattern,"\n</section>\n<section>\n",str(soup1))

In [17]:
soup2=BeautifulSoup(new_text,"html.parser")

In [18]:
update_html(soup2)

In [19]:
last_section=soup2.find_all("section")[-1]
print(last_section)
last_section.decompose()

<section>
<a name="section9"></a>
</section>


In [20]:
soup2.div.unwrap()
soup2.div.unwrap()

<div></div>

In [21]:
update_html(soup2)

In [22]:
for tag in soup2.div.find_all("div",recursive=False):
    tag.div.unwrap()


In [23]:
soup2.div.find_all("div",recursive=False)[0].div.unwrap()

<div></div>

In [24]:
soup2.div.div.name="section"

In [25]:
dir(soup2.findAll("section")[1].find("img"))

['DEFAULT_INTERESTING_STRING_TYPES',
 '__bool__',
 '__call__',
 '__class__',
 '__contains__',
 '__copy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '__weakref__',
 '_all_strings',
 '_find_all',
 '_find_one',
 '_is_xml',
 '_lastRecursiveChild',
 '_last_descendant',
 '_namespaces',
 '_should_pretty_print',
 'append',
 'attrs',
 'can_be_empty_element',
 'cdata_list_attributes',
 'childGenerator',
 'children',
 'clear',
 'contents',
 'decode',
 'decode_contents',
 'decompose',
 'decomposed',
 'default',
 'descendants',
 'encode',
 'encode_contents',
 'extend',
 'extract',
 'fetchNextSiblings'

In [26]:
update_html(soup2)

In [27]:
for section in soup2.findAll("section"):
    image=section.find("img")
    if image:
        image.attrs["alt"]=image.attrs.get("alt",image.find_previous("h3").text)
        parent=image.find_parent()
        if parent.name in ['p','li']:
            ul=image.find_previous("ul")
            ul.insert_before(image.extract())
            if parent.text:
                new_li=soup2.new_tag('li')
                new_li.append(soup2.new_string(str(parent.text)))
                ul.insert(0,new_li)
            parent.decompose()


In [28]:
for tag in soup2.findAll("strong"):
    tag.name="h4"

In [29]:
update_html(soup2)


In [30]:
def get_next_tag(soup):
    next_ele=soup
    while next_ele is not None:
        next_ele=next_ele.next_element
        if isinstance(next_ele,Tag):
            return next_ele
    return None

In [31]:
def get_previous_tag(soup):
    next_ele=soup
    while next_ele is not None:
        next_ele=next_ele.previous_element
        if isinstance(next_ele,Tag):
            return next_ele
    return None

In [32]:
soup2.select("section ul + *")

[<p><h4>Survival and spread:</h4></p>,
 <p><h4>Favorable conditions:</h4></p>,
 <p><h4>Survival and spread:</h4></p>,
 <p><h4>Favourable conditions:</h4></p>,
 <p><h4>Survival and spread:</h4></p>,
 <p><h4>Favourable conditions:</h4></p>,
 <p><h4>Survival and spread:</h4></p>,
 <p><h4>Favourable conditions:</h4></p>,
 <p><h4>Survival and spread:</h4></p>,
 <p><h4>Favourable conditions:</h4></p>,
 <p><h4>Survival and spread:</h4></p>,
 <p><h4>Favourable conditions:</h4></p>,
 <p><h4>Survival and spread:</h4></p>,
 <p><h4>Favourable conditions:</h4></p>]

In [33]:
for li in soup2.findAll("li"):
    for h4 in li.find_all("h4"):
        h4.extract()
        li.string=str(h4.text+" "+li.text)
        

In [34]:
update_html(soup2)

In [35]:
soup3=clone_soup(soup2)

In [36]:


update_html(soup3)

In [37]:
main_dict={}
for section in soup3.findAll("section")[1:]:
    disease=str(section.find("h3").text)
    d=dict()
    for h4 in section.findAll("h4"):
        content={}
        next_ele=get_next_tag(h4)
        if next_ele.name=="img":
            content["img"]={
                "src":next_ele.attrs["src"],
                "alt":next_ele.attrs.get("alt",disease)
            }
            next_ele=get_next_tag(next_ele)
        if next_ele.name=="ul":
            content["content"]=list(map(lambda x:x.text,next_ele.find_all("li")))
        d[str(h4.text)]=content

    main_dict[disease]=d

     

In [38]:
import json
json_object = json.dumps(main_dict, indent = 4) 
print(json_object)


{
    "Mint rust": {
        "Disease symptoms:": {
            "img": {
                "src": "https://static.vikaspedia.in/media/images_en/agriculture/crop-production/integrated-pest-managment/ipm-for-vegetables/ipm-strategies-for-mint/mint-rusttest.png",
                "alt": "Mint rust"
            },
            "content": [
                "Pale and distorted shoots in spring",
                "Dusty orange pustules on the stems and leaves.",
                "Pustule may change to dusty yellow or black in colour in later stage",
                "Large areas of leaf tissue die and plants may lose leaves"
            ]
        },
        "Survival and spread:": {
            "content": [
                "When the orange spore pustules mature and break open in June or July, the spores are spread to other plants by the wind.",
                "The fungus enters the plant through the leaves lNewly infected plants seldom show symptoms until the following spring."
            ]
      

In [41]:
soup3.findAll()

[<div><section><div><h3>Mint Diseases</h3></div><div><ol><li><a href="#section1">Mint rust</a></li><li><a href="#section2">Verticillium wilt</a></li><li><a href="#section3">Anthracnose</a></li><li><a href="#section4">Powdery mildew</a></li><li><a href="#section5">Black Stem Rot</a></li><li><a href="#section6">Stem and stolon canker</a></li><li><a href="#section7">Septoria leaf spot</a></li><li><a href="#section8">Disease cycles:</a></li><li><a href="#section9">IPM for Mint</a></li></ol></div></section><div>
 
 <section>
 <a name="section1"></a><h3>Mint rust</h3>
 <p><h4>Disease symptoms:</h4></p>
 <img alt="Mint rust" src="https://static.vikaspedia.in/media/images_en/agriculture/crop-production/integrated-pest-managment/ipm-for-vegetables/ipm-strategies-for-mint/mint-rusttest.png"/><ul>
 <li>Pale and distorted shoots in spring</li>
 <li>Dusty orange pustules on the stems and leaves.</li>
 <li>Pustule may change to dusty yellow or black in colour in later stage</li>
 <li>Large areas of 

In [55]:
for tag in soup3.findAll(["h4","ul"]):
        while not tag.parent.name == "section":
            tag.parent.unwrap()

In [59]:
BeautifulSoup(soup3.findAll("h4")[2].findNextSiblings()[0],"html.parser")

TypeError: 'NoneType' object is not callable

In [63]:
soup3.findAll("h4")[2]

[<ul>
 <li>Dry weather with high relative humidity</li>
 <li>Pathogens overwinter on mint stubble &amp; on wild mint.</li>
 <li>It spreads through propagation material and air</li>
 </ul>]